In [1]:
# Import Libs
import numpy as np
import pandas as pd

# ML Model Imports
from xgboost import XGBClassifier

# Tools Import
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV

# Metrics Import
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, average_precision_score

In [2]:
# ===== DATA LOADING =====
advanced = pd.read_csv('/Users/ericpark/NBA_MVP/Advanced.csv')
advanced_df = advanced[advanced['season'] >= 2010].copy()

awards = pd.read_csv('/Users/ericpark/NBA_MVP/Player_Award_Shares.csv')
awards_df = awards[awards['season'] >= 2010].copy()

per_game = pd.read_csv('/Users/ericpark/NBA_MVP/Player_Per_Game.csv')
per_game_df = per_game[per_game['season'] >= 2010].copy()

team_stats = pd.read_csv('/Users/ericpark/NBA_MVP/Team_Summaries.csv')
team_stats_df = team_stats[team_stats['season'] >= 2010][['season', 'abbreviation', 'playoffs', 'w', 'l']].dropna().copy()

team_stats_df['win_pct'] = ((team_stats_df['w']) / (team_stats_df['w'] + team_stats_df['l']))

In [3]:
# ===== MVP DATA PREPARATION =====
mvp_df = awards_df[awards_df['award'] == 'nba mvp'][['season', 'player_id', 'share', 'winner']].copy()
mvp_df['mvp_rank'] = mvp_df.groupby('season')['share'].rank(method='max', ascending=False)
mvp_df['winner_int'] = mvp_df['winner'].astype(int)

In [4]:
# ===== FEATURE ENGINEERING =====
dup_cols = ['lg', 'player', 'age', 'pos', 'g', 'gs']
full_stats = per_game_df.merge(advanced_df.drop(columns=dup_cols), how='inner', on=['season', 'player_id', 'team'])

# Add team wins/losses
full_stats = full_stats.merge(team_stats_df.drop(columns=['w', 'l']), how='left', left_on=['season', 'team'], right_on=['season', 'abbreviation'])

In [5]:
#df of traded players
traded_players = full_stats.merge(full_stats.loc[full_stats['team'].isin(['2TM', '3TM']), ['player_id', 'season']], on = ['player_id', 'season'], how = 'inner')#[['season', 'player_id', 'team', 'g', 'win_pct']]
traded_players['win_pct_rel'] = traded_players['win_pct'] * traded_players['g']

In [6]:
#adding relative win pct to players who got traded
traded_group = traded_players.loc[~traded_players['team'].isin(['2TM','3TM']),['season', 'player_id', 'team', 'g', 'win_pct', 'win_pct_rel']].groupby(['player_id', 'season']).sum()
traded_winpct = (traded_group['win_pct_rel']/traded_group['g']).reset_index()
#traded_winpct['team'] = '2TM'
traded_winpct.rename(columns = {0: 'win_pct'}, inplace = True)

In [7]:
#adding relative win pct back to main df
full_stats_idx = full_stats.set_index(['player_id', 'season'])
full_stats_idx.update(traded_winpct.set_index(['player_id', 'season']))
full_stats = full_stats_idx.reset_index()

In [8]:
# Filter for meaningful minutes
full_stats = full_stats[(full_stats['g'] >= 50) & (full_stats['mp_per_game'] >= 25)].copy()
#full_stats = full_stats.fillna(0)

# Initialize MVP columns
full_stats['winner'] = False
full_stats['share'] = 0
full_stats['mvp_rank'] = np.nan
full_stats['winner_int'] = 0

# Example interaction features
full_stats['usg_x_win'] = full_stats['usg_percent'] * full_stats['win_pct']
full_stats['vorp_x_win'] = full_stats['vorp'] * full_stats['win_pct']
full_stats['per_x_win'] = full_stats['per'] * full_stats['win_pct']

# Separate current season from training data
full_stats_2025 = full_stats[full_stats['season'] == 2025].copy()
full_stats_rest = full_stats[full_stats['season'] != 2025].copy()

# Update with MVP data
full_stats_rest_idx = full_stats_rest.set_index(['season', 'player_id'])
mvp_df_idx = mvp_df.set_index(['season', 'player_id'])
full_stats_rest_idx.update(mvp_df_idx)
full_stats_mvp = full_stats_rest_idx.reset_index()

# Handle traded players (keep '2TM' aggregate stats)
full_stats_mvp.sort_values(by='team', key=lambda col: col != '2TM', inplace=True)
full_stats_mvp.drop_duplicates(subset=['season', 'player'], keep='first', inplace=True)
full_stats_mvp.sort_values(by=['season', 'share'], ascending=False, inplace=True)

In [9]:
# ===== PREPARE FEATURES AND TARGET =====
# First, let's check correlation with MVP voting to decide between VORP and BPM
print("\n" + "="*70)
print("🔍 FEATURE CORRELATION ANALYSIS")
print("="*70)

# Check all features before dropping
all_feature_cols = full_stats_mvp.drop(['season', 'pos', 'player', 'player_id', 
                                         'winner', 'mvp_rank', 'lg', 'team', 'abbreviation', 'winner_int'], axis=1).columns

# Get top 5 MVP candidates per season to check correlation with voting
# Top 5 balances sample size with signal quality (serious contenders only)
top_candidates = full_stats_mvp[full_stats_mvp['mvp_rank'] <= 5].copy()

print(f"\nAnalyzing top 5 MVP candidates per season ({len(top_candidates)} players total)")

if len(top_candidates) > 0:
    print("\n📊 Correlation with MVP Vote Share (Top candidates only):")
    print("-" * 70)
    
    # Check key advanced stats
    key_stats = ['vorp', 'bpm', 'per', 'ws', 'ws_48']
    correlations = {}
    
    for stat in key_stats:
        if stat in top_candidates.columns:
            corr = top_candidates[stat].corr(top_candidates['share'])
            correlations[stat] = corr
            print(f"  {stat.upper():15s}: {corr:.4f}")
    
    # Find best predictor
    best_stat = max(correlations.items(), key=lambda x: x[1])
    print(f"\n✅ Best predictor of MVP voting: {best_stat[0].upper()} (r = {best_stat[1]:.4f})")


🔍 FEATURE CORRELATION ANALYSIS

Analyzing top 5 MVP candidates per season (75 players total)

📊 Correlation with MVP Vote Share (Top candidates only):
----------------------------------------------------------------------
  VORP           : 0.6350
  BPM            : 0.6196
  PER            : 0.6113
  WS             : 0.5783
  WS_48          : 0.6171

✅ Best predictor of MVP voting: VORP (r = 0.6350)


In [10]:
# Check multicollinearity among advanced stats
print("\n🔗 Multicollinearity Check (Advanced Stats):")
print("-" * 70)

advanced_stats = ['vorp', 'bpm', 'per', 'ws', 'ws_48']
existing_advanced = [s for s in advanced_stats if s in full_stats_mvp.columns]

if len(existing_advanced) > 1:
    corr_matrix = full_stats_mvp[existing_advanced].corr()
    
    print("\nCorrelation Matrix:")
    print(corr_matrix.round(3).to_string())
    
    print("\n⚠️  Highly Correlated Pairs (r > 0.80):")
    for i in range(len(existing_advanced)):
        for j in range(i+1, len(existing_advanced)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > 0.80:
                print(f"  {existing_advanced[i].upper()} <-> {existing_advanced[j].upper()}: {corr_val:.3f}")

# Remove highly correlated features based on correlation analysis
# Keeping: basic stats (pts, trb, ast), one advanced metric (vorp), efficiency (ts_percent), and team success (win_pct, playoffs)
print("\n📉 Removing redundant features to reduce multicollinearity...")


#features to keep
feature_cols = ['trb_per_game', 'ast_per_game', 'pts_per_game',  'ts_percent', 'usg_percent', 'vorp', 'per', 'usg_x_win', 'vorp_x_win', 'per_x_win']

X = full_stats_mvp[feature_cols]
y = full_stats_mvp['winner_int']

print(f"\n✅ Final streamlined features: {list(feature_cols)}")
print(f"\nDataset shape: {X.shape}")
print(f"Reduced from {len(all_feature_cols)} to {len(feature_cols)} features")
print(f"\nClass distribution: {y.value_counts().to_dict()}")
print(f"Class imbalance ratio: {(y == 0).sum() / (y == 1).sum():.1f}:1")
print("="*70)


🔗 Multicollinearity Check (Advanced Stats):
----------------------------------------------------------------------

Correlation Matrix:
        vorp    bpm    per     ws  ws_48
vorp   1.000  0.973  0.855  0.909  0.854
bpm    0.973  1.000  0.874  0.867  0.886
per    0.855  0.874  1.000  0.813  0.829
ws     0.909  0.867  0.813  1.000  0.940
ws_48  0.854  0.886  0.829  0.940  1.000

⚠️  Highly Correlated Pairs (r > 0.80):
  VORP <-> BPM: 0.973
  VORP <-> PER: 0.855
  VORP <-> WS: 0.909
  VORP <-> WS_48: 0.854
  BPM <-> PER: 0.874
  BPM <-> WS: 0.867
  BPM <-> WS_48: 0.886
  PER <-> WS: 0.813
  PER <-> WS_48: 0.829
  WS <-> WS_48: 0.940

📉 Removing redundant features to reduce multicollinearity...

✅ Final streamlined features: ['trb_per_game', 'ast_per_game', 'pts_per_game', 'ts_percent', 'usg_percent', 'vorp', 'per', 'usg_x_win', 'vorp_x_win', 'per_x_win']

Dataset shape: (2211, 10)
Reduced from 53 to 10 features

Class distribution: {0: 2196, 1: 15}
Class imbalance ratio: 146.4:1


In [18]:
# ===== HYPERPARAMETER TUNING =====
param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2, 0.3],
    'scale_pos_weight': [1, 10, 20, 30]
}

# Set up cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Base model
base_model = XGBClassifier(
    eval_metric='logloss',
    random_state=42
)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='average_precision',
    cv=skf,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

print("\n🔍 Starting hyperparameter tuning...")
random_search.fit(X, y)


🔍 Starting hyperparameter tuning...
Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='logloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           imp...
                                           random_state=42, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6, 7],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [100, 200, 300, 500],
                                        'scale_pos_weight': [1, 10, 20, 30],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   random_state=42, scoring='average_precision', verbose=2)

In [19]:
# ===== RESULTS =====
print("\n✅ Best parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\n📊 Best average precision score: {random_search.best_score_:.4f}")


✅ Best parameters found:
  subsample: 0.8
  scale_pos_weight: 10
  n_estimators: 200
  min_child_weight: 3
  max_depth: 4
  learning_rate: 0.1
  gamma: 0
  colsample_bytree: 0.6

📊 Best average precision score: 0.8678


In [20]:
# ===== EVALUATE BEST MODEL =====
best_model = random_search.best_estimator_

# Cross-validation with best model
cv_scores = cross_val_score(best_model, X, y, cv=skf, scoring='average_precision')
print(f"\n🎯 Cross-validation scores: {cv_scores}")
print(f"   Mean: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Train/test split evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
y_pred_prob = best_model.predict_proba(X_test)[:, 1]

print("\n📈 Test Set Performance:")
print(f"  Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"  ROC AUC: {roc_auc_score(y_test, y_pred_prob):.4f}")
print(f"  Average Precision: {average_precision_score(y_test, y_pred_prob):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


🎯 Cross-validation scores: [0.86666667 0.72222222 0.91666667 0.91666667 0.91666667]
   Mean: 0.8678 (+/- 0.1506)

📈 Test Set Performance:
  Accuracy: 0.9955
  ROC AUC: 0.9977
  Average Precision: 0.7556

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       440
           1       0.67      0.67      0.67         3

    accuracy                           1.00       443
   macro avg       0.83      0.83      0.83       443
weighted avg       1.00      1.00      1.00       443



In [21]:
# ===== FEATURE IMPORTANCE =====
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🔝 Top 15 Most Important Features:")
print(feature_importance.to_string(index=False))


🔝 Top 15 Most Important Features:
     feature  importance
  vorp_x_win    0.476502
         per    0.262348
        vorp    0.113620
   per_x_win    0.077729
   usg_x_win    0.022323
 usg_percent    0.014004
ast_per_game    0.012607
trb_per_game    0.011450
  ts_percent    0.009418
pts_per_game    0.000000


In [22]:
# ===== TEST ON PAST SEASON =====
sample_season = full_stats_mvp[full_stats_mvp['season'] == 2024].copy()
if len(sample_season) > 0:
    print(f"\n{'='*60}")
    print("🏀 2024 Season - Model Validation:")
    print(f"{'='*60}")
    
    X_sample = sample_season[feature_cols]
    sample_probs = best_model.predict_proba(X_sample)[:, 1]
    sample_season['mvp_probability'] = sample_probs
    
    # Rank by probability
    sample_season_ranked = sample_season.sort_values('mvp_probability', ascending=False)
    
    print("\n📊 Top 10 Predicted MVP Candidates:")
    top_10 = sample_season_ranked.head(10)[['player', 'team', 'win_pct', 'mvp_probability', 'winner_int', 'mvp_rank', 'share']]
    top_10_display = top_10.copy()
    top_10_display['win_pct'] = (top_10_display['win_pct'] * 100).round(1).astype(str) + '%'
    top_10_display['mvp_probability'] = (top_10_display['mvp_probability'] * 100).round(1).astype(str) + '%'
    top_10_display.columns = ['Player', 'Team', 'Win%', 'MVP Prob', 'Actual Winner', 'Actual Rank', 'Actual Share']
    print(top_10_display.to_string(index=False))
    
    # Check if we got the top 3 right
    actual_top_3 = set(sample_season[sample_season['mvp_rank'] <= 3]['player'].values)
    predicted_top_3 = set(sample_season_ranked.head(3)['player'].values)
    overlap = actual_top_3.intersection(predicted_top_3)
    
    print(f"\n✓ Correctly predicted in top 3: {len(overlap)}/3")
    if overlap:
        print(f"  Players: {', '.join(overlap)}")
    
    # Show actual vs predicted winner
    actual_winner = sample_season[sample_season['winner_int'] == 1]['player'].values[0]
    predicted_winner = sample_season_ranked.iloc[0]['player']
    print(f"\n🏆 Actual winner: {actual_winner}")
    print(f"🔮 Predicted winner: {predicted_winner}")
    if actual_winner == predicted_winner:
        print("   ✅ CORRECT!")
    else:
        winner_rank = sample_season_ranked[sample_season_ranked['player'] == actual_winner].index[0]
        winner_position = sample_season_ranked.index.get_loc(winner_rank) + 1
        print(f"   ❌ Actual winner was ranked #{winner_position} by model")


🏀 2024 Season - Model Validation:

📊 Top 10 Predicted MVP Candidates:
                 Player Team  Win% MVP Prob  Actual Winner  Actual Rank  Actual Share
           Nikola Jokić  DEN 69.5%    97.0%              1          1.0         0.935
Shai Gilgeous-Alexander  OKC 69.5%    12.3%              0          2.0         0.646
  Giannis Antetokounmpo  MIL 59.8%    11.8%              0          4.0         0.194
            Luka Dončić  DAL 61.0%     5.5%              0          3.0         0.572
       Domantas Sabonis  SAC 56.1%     0.4%              0          8.0         0.003
           LeBron James  LAL 57.3%     0.3%              0          NaN         0.000
           James Harden  LAC 62.2%     0.3%              0          NaN         0.000
             Trae Young  ATL 43.9%     0.3%              0          NaN         0.000
      Tyrese Haliburton  IND 57.3%     0.3%              0          NaN         0.000
          Fred VanVleet  HOU 50.0%     0.3%              0          N

In [23]:
# ===== PREDICT FOR 2025 SEASON =====
if len(full_stats_2025) > 0:
    print(f"\n{'='*60}")
    print("🔮 2025 Season Predictions:")
    print(f"{'='*60}")
    
    X_2025 = full_stats_2025[feature_cols]
    probs_2025 = best_model.predict_proba(X_2025)[:, 1]
    full_stats_2025['mvp_probability'] = probs_2025
    
    # Rank by probability
    full_stats_2025_ranked = full_stats_2025.sort_values('mvp_probability', ascending=False)
    
    print("\n🏆 Predicted Top 3 MVP Candidates:")
    top_3 = full_stats_2025_ranked.head(3)
    for i, (idx, row) in enumerate(top_3.iterrows(), 1):
        medals = ['🥇', '🥈', '🥉']
        positions = ['1st', '2nd', '3rd']
        win_pct_str = f"{row['win_pct']*100:.1f}%" if not pd.isna(row['win_pct']) else "N/A"
        print(f"  {medals[i-1]} {positions[i-1]}: {row['player']} ({row['team']}) - {row['mvp_probability']:.1%} | Win%: {win_pct_str}")
    
    print("\n📊 Top 15 MVP Candidates:")
    top_15 = full_stats_2025_ranked.head(15)[['player', 'team', 'win_pct', 'playoffs', 'mvp_probability']]
    top_15_display = top_15.copy()
    top_15_display['win_pct'] = top_15_display['win_pct'].apply(lambda x: f"{x*100:.1f}%" if not pd.isna(x) else "N/A")
    top_15_display['playoffs'] = top_15_display['playoffs'].apply(lambda x: "Yes" if x == 1 else "No" if x == 0 else "N/A")
    top_15_display['mvp_probability'] = (top_15_display['mvp_probability'] * 100).round(1).astype(str) + '%'
    top_15_display.columns = ['Player', 'Team', 'Win%', 'Playoffs', 'MVP Prob']
    print(top_15_display.to_string(index=False))


🔮 2025 Season Predictions:

🏆 Predicted Top 3 MVP Candidates:
  🥇 1st: Nikola Jokić (DEN) - 95.8% | Win%: 61.0%
  🥈 2nd: Shai Gilgeous-Alexander (OKC) - 94.3% | Win%: 82.9%
  🥉 3rd: Giannis Antetokounmpo (MIL) - 1.5% | Win%: 58.5%

📊 Top 15 MVP Candidates:
                 Player Team  Win% Playoffs MVP Prob
           Nikola Jokić  DEN 61.0%      Yes    95.8%
Shai Gilgeous-Alexander  OKC 82.9%      Yes    94.3%
  Giannis Antetokounmpo  MIL 58.5%      Yes     1.5%
       Donovan Mitchell  CLE 78.0%      Yes     0.4%
            Luka Dončić  2TM 55.1%      N/A     0.3%
        Cade Cunningham  DET 53.7%      Yes     0.3%
      Tyrese Haliburton  IND 61.0%      Yes     0.3%
             Trae Young  ATL 48.8%      Yes     0.3%
           James Harden  LAC 61.0%      Yes     0.3%
           LeBron James  LAL 61.0%      Yes     0.3%
          Jarrett Allen  CLE 78.0%      Yes     0.2%
            Jalen Duren  DET 53.7%      Yes     0.1%
            Rudy Gobert  MIN 59.8%      Yes     0.1%


In [24]:
#does a good job of predicting real contenders for MVP (Top 3/4)
#MVP prob is very top heavy

#